# Home assignment 2

You should work on the assignement in groups of 2 participants. 

Upload your solution as a jupyter notebook to L2P by 26th of June 23:59h. (The deadline is strict)

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results.



# Group
Deniz Schmidt 334744
Dinc Erduran 262999

# 1. Dynamic PageRank
Consider a random walk setting where the transistion matrix changes over time. At any point of time the probability of a random surfer to jump to a linked page is proportional to the number of previous visits. To start with all the pages are equally likely to be chosen but as the walk continues and the nodes are visited the transition probability changes as proportional to number of previous visits. For example let a page 'a' is linked to pages 'b', 'c' and 'd'. The random surfer currently resides at 'a' and the pages 'b', 'c' and 'd' have already been visited 5, 3 and 2 times respectively. The transition probability would be 0.5, 0.3 and 0.2 respectively. As a new node is viited the probabilities change. The random surfer continues to surf with probability 0.8. Generate 100 random walks and rank the nodes based on the frequency of visit. The random walk should be performed on a drected Erdos-Renyi graph with number of nodes n=200 and probability of edge creation p = 0.4. 

Hint: Use networkx library for generating graph.
    

In [ ]:
from networkx import nx
import random
from collections import defaultdict

## generate (directed) GNP random graph
G = nx.MultiDiGraph(nx.gnp_random_graph(200, 0.4, directed=True))
visit_freq_map = defaultdict(int) # to count node visits

for i in range(100):
    H = G.copy()
    currNode = 0
    while(random.random() >= 0.2):
        new_link = list(H.edges(currNode))[random.randint(0, len(H.edges(currNode))-1)]
        H.add_edge(new_link[0], new_link[1])  # to increase visit probability
        currNode = new_link[1]
        visit_freq_map[currNode] += 1

# print out nodes by frequency of visit in descending order
print("nodes by frequency of visit:")
print(sorted(visit_freq_map.items(), key=lambda kv: kv[1], reverse=True))

# 2. Recommendation
a. Compare the recommendation algorithms (SVD, NMF, Baseline, k-NN and Random) available in surprise package on movielens dataset in terms of RMSE and MAE.

b. Consider the movielens dataset and divide it into (i) training set with 50% of the data (train the algorithms on this part) and (ii) 25% validation set and (iii) test set with the rest. Estimate the ratings of the test set using the algorithms (same as in a) provided by the package on the training set. Your final rating should be weighted average of the ratings predicted by the algorithms. The weights should be learnt on the validation set. Performance should be measured in terms of RMSE.

Hint: Use grid search/step-wise update like SGD for learning the weights

In [ ]:
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import NMF

data = Dataset.load_builtin('ml-100k')

# 5-fold cross validation results for 5 algorithms 
cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(NMF(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(surprise.prediction_algorithms.baseline_only.BaselineOnly(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(surprise.prediction_algorithms.knns.KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(surprise.prediction_algorithms.random_pred.NormalPredictor(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Comparison on RMSE, MAE with 5-fold cv result: SVD > Baseline > NMF > k-NN > Random

# 3. Hidden Markov model
Consider the HMM package https://hmmlearn.readthedocs.io/en/latest/

a. Generate sequences with multinomial HMM (2 symbols and 4 hidden states) and given parameters. Start probability - {0.4,0.2,0.1,0.3}, Transition matrix - {{0.2,0.3,0.1,0.4},{0.3,0.3,0.2,0.2},{0.4,0.2,0.3,0.1},{0.2,0.3,0.1,0.4}}, Emission probability - {{0.2,0.8},{0.1,0.9},{0.5,0.5},{0.6,0.4}}.


b. Consider a sequence - {1 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1}. Fit a multinomial HMM considering 4 states and obtain hidden state which is most likely to have generated the symbol


In [ ]:
import numpy as np
from hmmlearn import hmm

#a)
model = hmm.MultinomialHMM(n_components=4)
model.n_features = 2
model.startprob_ = np.array([0.4,0.2,0.1,0.3])
model.transmat_ = np.array([[0.2,0.3,0.1,0.4],
                           [0.3,0.3,0.2,0.2],
                           [0.4,0.2,0.3,0.1],
                           [0.2,0.3,0.1,0.4]])
model.emissionprob_ = np.array([[0.2,0.8],
                               [0.1,0.9],
                               [0.5,0.5],
                               [0.6,0.4]])

X,Y = model.sample(10)
print(X)
print(Y)

#b)
seq = [[1],[0],[0],[0],[1],[1],[1],[1],[0],[1],[0],[1],[0],[1],[0],[1],[1],[1],[0],[0],[0],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1]]
remodel = hmm.MultinomialHMM(n_components=4)
remodel.fit(seq, [len(seq)])
Z = remodel.predict(seq, [len(seq)])

print(Z)

# 4. PrefixSpan 
Implement the Prefix algorithm for Sequential Pattern Mining.

In [ ]:
#in: array of strings, int
#out: array of frequent sequences
def prefixSpan(database, min_sup):
    symbols = dict()
    frequents = []
    projected = dict()

    #mine frequent singles
    for seq in database:
        for sym in seq:
            if sym not in symbols:
                symbols[sym] = 1
            else:
                symbols[sym] += 1

    keys = list(symbols.keys())
    for key in keys:
        if symbols[key] >= min_sup:
            frequents.append(key)
    singleFrequents = frequents.copy()

    for sym in frequents:
        projected[sym] = []

    for seq in database:
        for sym in frequents:
            find = seq.find(sym)
            if find != -1 and len(seq) > find+1:
                projected[sym].append(seq[find+1:])

    #print(symbols)
    #print(frequents)
    #print(projected)

    #build every possible seq
    foundSeq = frequents.copy()
    while len(foundSeq) > 0:
        #print("Currently found frequents: ")
        #print(frequents)
        nextSeq = buildSequences(foundSeq, singleFrequents)
        #print("Now checking:")
        #print(nextSeq)
        foundSeq = []
        projected = buildProjectedDB(projected, frequents, singleFrequents)
        #print("Using prjected db:")
        #print(projected)
        for seq in nextSeq:
            if len(projected[seq]) >= min_sup:
                foundSeq.append(seq)
        frequents += foundSeq

    return frequents

#in: list,list
#out: list
def buildSequences(sequences, symbols):
    ret = []
    for seq in sequences:
        for sym in symbols:
            ret.append(seq+sym)
    return ret

#in: dict, list, list
#out: dict
def buildProjectedDB(database, latestSequences, singleFrequents):
    newDB = database.copy()
    #for all frequent sequences found until now
    for frSeq in latestSequences:
        #for each possible next frequent single appended to a frequent sequence
        for sym in singleFrequents:
            newDB[frSeq+sym] = []
            for prSeq in database[frSeq]:
                #this allows sequences with symbols inbetween; amusing
                #find = prSeq.find(sym)
                #if find != -1 and len(prSeq) > find+1:
                #    newDB[frSeq+sym].append(prSeq[find+1:])
                if prSeq[0] == sym:
                    newDB[frSeq+sym].append(prSeq[1:])
    return newDB

#test
db = ["612735769345734","347153456","7932465","17253476845","623452386"]
print("Frequent (>=2) sequences found:")
print(prefixSpan(db, 2))